# Convergence of weighted Jacobi

$$e_{k+1} = Me_k = (I-P^{-1}A)e_k = (I-wD^{-1}A)e_k$$

$$M = 
\begin{bmatrix}
1 &0 &\cdots &0\\
0 &1 &\cdots &0\\
\vdots &\vdots &\ddots &\vdots\\
0 &0 &\cdots &1
\end{bmatrix}
- w* \begin{bmatrix}
1/2 &0 &\cdots &0\\
0 &1/2 &\cdots &0\\
\vdots &\vdots &\ddots &\vdots\\
0 &0 &\cdots &1/2
\end{bmatrix}
\begin{bmatrix}
2 &-1 &\cdots &0\\
-1 &2 &\cdots &0\\
\vdots &\vdots &\ddots &\vdots\\
0 &0 &\cdots &2
\end{bmatrix}
=\begin{bmatrix}
1-w &w/2 &\cdots &0\\
w/2 &1-w &\cdots &0\\
\vdots &\vdots &\ddots &\vdots\\
0 &0 &\cdots &1-w
\end{bmatrix}$$

The eigenvalues take the form $\lambda_j(M) = 1-\frac{w}{2}(2-2cos j\theta) = 1-w+wcos j\theta$, where $\theta = \frac{\pi}{N+1}, j\in\{ 1,2,\cdots, N+1\}$.

Consider that $M$ is a $5\times 5$ matrix and $w = 2/3$

$$M = \begin{bmatrix}
1/3 &1/3 &0 &0 &0\\
1/3 &1/3 &1/3 &0 &0\\
0 &1/3 &1/3 &1/3 &0\\
0 &0 &1/3 &1/3 &1/3\\
0 &0 &0 &1/3 &1/3 \\
\end{bmatrix}$$

The eigenvalues are $$.

In [102]:
import numpy as np
import scipy.sparse.linalg as alg


def gen_matrices(d):
    A = np.zeros((d, d))
    for i in range(d):
        A[i][i] = 2
        if i != 0:
            A[i][i-1] = -1
        if i != d-1:
            A[i][i+1] = -1
    row = d//2
    R = np.zeros((row, d))
    for i in range(row):
        R[i][2*i] = 1
        R[i][2*i+1] = 2
        R[i][2*i+2] = 1
    R = 1/4*R
    I = 2*R.T
    b = np.random.uniform(-10, 10, d)
    return A, b, R, I
    
# A = np.array([[ 2, -1,  0,  0,  0],
#               [-1,  2, -1,  0,  0],
#               [ 0, -1,  2, -1,  0],
#               [ 0,  0, -1,  2, -1],
#               [ 0,  0,  0, -1,  2]])

# R = 1/4*np.array([[1, 2, 1, 0, 0],
#                   [0, 0, 1, 2, 1]])
# I = 2*R.T

# b = np.random.uniform(-10, 10, 5)

def CustomJacobi(n, A, b, x=None, w=1):
    if x is None:
        x = np.zeros(b.shape)
    I = np.identity(len(x))
    P = A * np.identity(len(x))/w
    invP = np.linalg.inv(P)
    M = I - invP @ A
    print("eigenvalue of M is\n {}\n".format(np.linalg.eig(M)[0]))
    for i in range(n):
        x = M @ x + invP @ b
    return x

def multigrid(n, A, b, R, I, w):
    u = CustomJacobi(n//2, A, b, None, w)
    r = b - A @ u
    r2h = R @ r
    A2h = R @ A @ I
    E2h = CustomJacobi( n//4, A2h, r2h, None, w)
    E = I @ E2h
    u = u + E
    u = CustomJacobi(n//4, A, b, u, w)
    return u
    
        

def main():
    d = 9
    A, b, R, I = gen_matrices(d)
    
    n = d*d
    X = np.linalg.solve(A, b)
    print("x is \n{}\n".format(X))
    x = CustomJacobi(n, A, b)
    error = np.sqrt(np.sum((x-X)**2))
    print("Jacobi approximation x is \n{}\n error is {}\n".format(x, error))
    
    x = multigrid(n, A, b, R, I, 1)
    error = np.sqrt(np.sum((x-X)**2))
    print("multigrid approximation x is \n{}\n error is {}\n".format(x, error))
    
    x = multigrid(n, A, b, R, I, 2/3)
    error = np.sqrt(np.sum((x-X)**2))
    print("multigrid approximation x is \n{}\n error is {}\n".format(x, error))
    
if __name__=="__main__":
    main()

x is 
[13.18448478 18.05034341 14.57189935  6.49660224  0.0341817  -1.01747314
  2.11025671 -3.98057737 -5.21057898]

eigenvalue of M is
 [-9.51056516e-01 -8.09016994e-01 -5.87785252e-01 -3.09016994e-01
 -2.26430432e-17  9.51056516e-01  8.09016994e-01  3.09016994e-01
  5.87785252e-01]

Jacobi approximation x is 
[13.15587735 17.98578377 14.49700434  6.39214285 -0.0583932  -1.12193216
  2.03536238 -4.04513641 -5.23918599]
 error is 0.22712231232811425

eigenvalue of M is
 [-9.51056516e-01 -8.09016994e-01 -5.87785252e-01 -3.09016994e-01
 -2.26430432e-17  9.51056516e-01  8.09016994e-01  3.09016994e-01
  5.87785252e-01]

eigenvalue of M is
 [-0.80901699 -0.30901699  0.80901699  0.30901699]

eigenvalue of M is
 [-9.51056516e-01 -8.09016994e-01 -5.87785252e-01 -3.09016994e-01
 -2.26430432e-17  9.51056516e-01  8.09016994e-01  3.09016994e-01
  5.87785252e-01]

multigrid approximation x is 
[13.164041   18.0480916  14.51837834  6.49295873 -0.03197089 -1.02111665
  2.05674059 -3.98282918 -5.2310